# Truck Analysis

# 

### Importing Neccesary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Suppressing warnings
import warnings  
warnings.filterwarnings('ignore')

# 

### Loading and Inspecting the Data

In [2]:
# Importing the CSV file into a DataFrame
kite = pd.read_csv('clustered_data.csv', index_col = 0)

# Viewing the DataFrame
kite.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label
0,26.84,2295440,7700,2023-04-18,MFS,375.40,0.00,2023-04-18,34.46,110.00,0.225000,0.200000,766,DY6,52.496759,-2.173682,1
1,26.84,2277415,67940,2023-03-28,DX Freight,236.61,0.00,2023-03-28,16.45,33.30,0.959000,0.473039,766,DY6,52.496759,-2.173682,1
2,26.84,2279524,67940,2023-03-30,DX Freight,70.58,13.20,2023-03-30,11.79,21.80,0.209100,0.222222,766,DY6,52.496759,-2.173682,1
3,26.84,2275231,483789,2023-03-27,DX Express,76.95,7.22,2023-03-27,6.81,17.77,0.040200,0.030342,766,DY6,52.496759,-2.173682,1
4,26.84,2285334,483789,2023-04-05,DX Express,147.30,0.00,2023-04-05,11.46,34.70,0.100552,0.052361,766,DY6,52.496759,-2.173682,1


In [3]:
# Checking the shape of the DataFrame
kite.shape

(4119, 17)

In [4]:
# Exploring the metadata
kite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4119 entries, 0 to 4118
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   distance        4119 non-null   float64
 1   OrderID         4119 non-null   int64  
 2   WebsiteID       4119 non-null   int64  
 3   OrderDate       4119 non-null   object 
 4   Courier         4119 non-null   object 
 5   ProductsExVAT   4119 non-null   float64
 6   DeliveryExVAT   4119 non-null   float64
 7   DateDespatched  4119 non-null   object 
 8   DeliveryCost    4119 non-null   float64
 9   Weight          4119 non-null   float64
 10  Volume (m3)     4119 non-null   float64
 11  Pallets         4119 non-null   float64
 12  id              4119 non-null   int64  
 13  Outward Code    4119 non-null   object 
 14  latitude        4119 non-null   float64
 15  longitude       4119 non-null   float64
 16  Cluster Label   4119 non-null   int64  
dtypes: float64(9), int64(4), object(4

In [5]:
# Changing the data types of OrderDate and DateDespatched to datetime
# Formatting the dates so they are yyyy/mm/dd
kite['OrderDate'] = pd.to_datetime(kite['OrderDate'], format = '%Y-%m-%d')
kite['DateDespatched'] = pd.to_datetime(kite['DateDespatched'], format = '%Y-%m-%d')

# Checking this has worked
print(kite.info())
kite.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4119 entries, 0 to 4118
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   distance        4119 non-null   float64       
 1   OrderID         4119 non-null   int64         
 2   WebsiteID       4119 non-null   int64         
 3   OrderDate       4119 non-null   datetime64[ns]
 4   Courier         4119 non-null   object        
 5   ProductsExVAT   4119 non-null   float64       
 6   DeliveryExVAT   4119 non-null   float64       
 7   DateDespatched  4119 non-null   datetime64[ns]
 8   DeliveryCost    4119 non-null   float64       
 9   Weight          4119 non-null   float64       
 10  Volume (m3)     4119 non-null   float64       
 11  Pallets         4119 non-null   float64       
 12  id              4119 non-null   int64         
 13  Outward Code    4119 non-null   object        
 14  latitude        4119 non-null   float64       
 15  long

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label
0,26.84,2295440,7700,2023-04-18,MFS,375.40,0.00,2023-04-18,34.46,110.00,0.225000,0.200000,766,DY6,52.496759,-2.173682,1
1,26.84,2277415,67940,2023-03-28,DX Freight,236.61,0.00,2023-03-28,16.45,33.30,0.959000,0.473039,766,DY6,52.496759,-2.173682,1
2,26.84,2279524,67940,2023-03-30,DX Freight,70.58,13.20,2023-03-30,11.79,21.80,0.209100,0.222222,766,DY6,52.496759,-2.173682,1
3,26.84,2275231,483789,2023-03-27,DX Express,76.95,7.22,2023-03-27,6.81,17.77,0.040200,0.030342,766,DY6,52.496759,-2.173682,1
4,26.84,2285334,483789,2023-04-05,DX Express,147.30,0.00,2023-04-05,11.46,34.70,0.100552,0.052361,766,DY6,52.496759,-2.173682,1


# 

### Only Looking at Working Days

In [6]:
# Creating a column to identify the day of the week that the order was dispatched
kite['weekday'] = kite['DateDespatched'].apply(lambda x: x.weekday())

# Removing orders from the DataFrame that were dispatched on a Saturday (5) or Sunday (6)    
kite_wd = kite.loc[~((kite['weekday'] == 5) | (kite['weekday'] == 6))]

# Checking this has worked
kite_wd['weekday'].unique()

array([1, 3, 0, 2, 4], dtype=int64)

In [7]:
# Removing orders that were dispatched on the Easter bank holidays April 7th & 10th
kite_wd = kite_wd[~kite_wd['DateDespatched'].isin([pd.Timestamp('20230407'), pd.Timestamp('20230410')])]

# Checking this has worked
kite_wd['DateDespatched'].unique()

array(['2023-04-18T00:00:00.000000000', '2023-03-28T00:00:00.000000000',
       '2023-03-30T00:00:00.000000000', '2023-03-27T00:00:00.000000000',
       '2023-04-05T00:00:00.000000000', '2023-04-14T00:00:00.000000000',
       '2023-04-21T00:00:00.000000000', '2023-04-04T00:00:00.000000000',
       '2023-04-13T00:00:00.000000000', '2023-04-19T00:00:00.000000000',
       '2023-04-17T00:00:00.000000000', '2023-03-31T00:00:00.000000000',
       '2023-04-11T00:00:00.000000000', '2023-04-20T00:00:00.000000000',
       '2023-04-03T00:00:00.000000000', '2023-04-24T00:00:00.000000000',
       '2023-03-29T00:00:00.000000000', '2023-04-12T00:00:00.000000000',
       '2023-04-06T00:00:00.000000000'], dtype='datetime64[ns]')

In [8]:
# Viewing the DataFrame
kite_wd.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
0,26.84,2295440,7700,2023-04-18,MFS,375.40,0.00,2023-04-18,34.46,110.00,0.225000,0.200000,766,DY6,52.496759,-2.173682,1,1
1,26.84,2277415,67940,2023-03-28,DX Freight,236.61,0.00,2023-03-28,16.45,33.30,0.959000,0.473039,766,DY6,52.496759,-2.173682,1,1
2,26.84,2279524,67940,2023-03-30,DX Freight,70.58,13.20,2023-03-30,11.79,21.80,0.209100,0.222222,766,DY6,52.496759,-2.173682,1,3
3,26.84,2275231,483789,2023-03-27,DX Express,76.95,7.22,2023-03-27,6.81,17.77,0.040200,0.030342,766,DY6,52.496759,-2.173682,1,0
4,26.84,2285334,483789,2023-04-05,DX Express,147.30,0.00,2023-04-05,11.46,34.70,0.100552,0.052361,766,DY6,52.496759,-2.173682,1,2


# 

### Only Looking to Replace DX Freight, MFS, and Split

In [9]:
# Removing DX Express from the DataFrame
kite_wd = kite_wd.loc[~(kite_wd['Courier'] == "DX Express")]

# Checking this has worked
kite_wd['Courier'].unique()

array(['MFS', 'DX Freight', 'Split'], dtype=object)

In [10]:
# Viewing the DataFrame
kite_wd.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
0,26.84,2295440,7700,2023-04-18,MFS,375.40,0.0,2023-04-18,34.46,110.00,0.2250,0.200000,766,DY6,52.496759,-2.173682,1,1
1,26.84,2277415,67940,2023-03-28,DX Freight,236.61,0.0,2023-03-28,16.45,33.30,0.9590,0.473039,766,DY6,52.496759,-2.173682,1,1
2,26.84,2279524,67940,2023-03-30,DX Freight,70.58,13.2,2023-03-30,11.79,21.80,0.2091,0.222222,766,DY6,52.496759,-2.173682,1,3
5,26.84,2292112,778040,2023-04-14,DX Freight,104.00,0.0,2023-04-14,20.55,44.20,0.1235,0.054167,766,DY6,52.496759,-2.173682,1,4
6,26.84,2298627,846996,2023-04-21,DX Freight,111.36,0.0,2023-04-21,23.25,51.12,0.1296,0.100000,766,DY6,52.496759,-2.173682,1,4


# 

### Writing a Function to Compute the Trucks Needed 

In [11]:
# Creating a function to identify trucks needed based on weight
def trucks_weight(x):
    if x.iloc[0] <= 1500:
        return 3.5
    elif 1500 < x.iloc[0] <= 4000:
        return 7.5
    elif 4000 < x.iloc[0] <= 10000:
        return 18
    else:
        return 1000 

In [12]:
# Creating a function to identify trucks needed based on volume
def trucks_volume(x):
    if x.iloc[1] <= 2.5:
        return 3.5
    elif 2.5 < x.iloc[1] <= 25:
        return 7.5
    elif 25 < x.iloc[1] <= 45:
        return 18
    else:
        return 1000

In [13]:
# Creating a function to identify trucks needed based on pallets
def trucks_pallet(x):
    if x.iloc[2] <= 2:
        return 3.5
    elif 2 < x.iloc[2] <= 14:
        return 7.5
    elif 14 < x.iloc[2] <= 24:
        return 18
    else:
        return 1000

In [14]:
# Creating a function to identify the number of trucks needed based on weight, volume, and pallets
def trucks_needed(x):
    truck = max(trucks_weight(x), trucks_volume(x), trucks_pallet(x))     
    if truck < 1000:
        print("The", truck, "- tonne truck is needed.")
    else:
        print("More investigation is needed.")

# 

### Investigating the Number of Trucks Needed in Each Cluster (Median)

In [15]:
# Calulating the number of orders each day for each cluster
clustero = kite_wd.pivot_table(index = 'Cluster Label', 
                               columns = 'DateDespatched', 
                               values = 'OrderID', aggfunc = 'count')
clustero

DateDespatched,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-03,2023-04-04,2023-04-05,2023-04-06,2023-04-11,2023-04-12,2023-04-13,2023-04-14,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-24
Cluster Label,,,,,,,,,,,,,,,,,,,
0,9.0,9.0,17.0,8.0,15.0,13.0,17.0,14.0,14.0,9.0,12.0,13.0,11.0,13.0,16.0,17.0,13.0,11.0,4.0
1,32.0,54.0,37.0,40.0,30.0,53.0,49.0,39.0,37.0,51.0,54.0,46.0,36.0,44.0,61.0,54.0,42.0,41.0,9.0
2,42.0,35.0,26.0,23.0,36.0,37.0,39.0,42.0,26.0,34.0,46.0,40.0,23.0,51.0,38.0,48.0,36.0,25.0,4.0
3,18.0,18.0,29.0,28.0,17.0,36.0,28.0,30.0,31.0,27.0,37.0,19.0,14.0,42.0,35.0,27.0,25.0,19.0,2.0
4,21.0,24.0,22.0,23.0,21.0,23.0,18.0,16.0,18.0,32.0,23.0,18.0,13.0,26.0,29.0,20.0,16.0,16.0,NaN


In [16]:
# Calculating the median number of orders on a single day for each cluster 
clustero1 = clustero.median(axis=1) 
clustero1

Cluster Label
0    13.0
1    42.0
2    36.0
3    27.0
4    21.0
dtype: float64

In [17]:
# On the basis that one truck can do a maximum of 25 deliveries in a day, working out
# how many trucks are needed for each cluster
trucks_per_cluster = (clustero1/25).apply(np.ceil)
trucks_per_cluster

Cluster Label
0    1.0
1    2.0
2    2.0
3    2.0
4    1.0
dtype: float64

# 

### Investigating the Weight, Volume, and Number of Pallets Used in Each Cluster (Median)

#### Total Weight of Orders in Each Cluster 

In [18]:
# Calulating the total weight of orders each day for each cluster
clusterw = kite_wd.pivot_table(index = 'Cluster Label', 
                               columns = 'DateDespatched', 
                               values = 'Weight', aggfunc = 'sum')
clusterw

DateDespatched,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-03,2023-04-04,2023-04-05,2023-04-06,2023-04-11,2023-04-12,2023-04-13,2023-04-14,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-24
Cluster Label,,,,,,,,,,,,,,,,,,,
0,861.03,453.91,2345.64,652.47,934.83,971.10,1934.39,2436.08,2117.81,1329.11,1197.31,1476.97,1686.32,1569.96,1437.16,2255.06,813.88,1035.22,376.21
1,6085.22,4655.93,3643.41,5503.92,6545.02,8274.88,8492.52,7210.06,3823.51,8140.87,5608.60,12033.06,6166.71,6288.21,13017.81,5936.15,4344.84,5983.70,1172.14
2,4469.25,5724.00,5086.67,3097.53,10774.05,4171.57,4052.52,4837.88,4569.78,4531.13,5784.69,4725.12,2360.12,4853.88,3259.96,4233.10,8112.37,5512.70,146.65
3,1901.76,2020.44,3506.48,3009.75,788.30,2841.85,3029.89,3824.49,4986.93,3528.53,5845.49,2238.81,1416.39,3710.51,2582.07,2770.10,2176.27,2545.50,93.44
4,1615.05,4781.02,2333.34,3407.72,2427.28,2212.35,1783.50,1002.20,1047.23,1696.83,2739.14,1525.73,3022.97,1993.01,3990.24,1776.91,854.24,2632.48,NaN


In [19]:
# Finding the median total weight of orders each day for each cluster
clusterw1 = clusterw.median(axis=1)
clusterw1

Cluster Label
0    1329.11
1    6085.22
2    4569.78
3    2770.10
4    2102.68
dtype: float64

In [20]:
# Adjusting this for the number of trucks needed
clusterw2 = clusterw1/trucks_per_cluster
clusterw2

Cluster Label
0    1329.11
1    3042.61
2    2284.89
3    1385.05
4    2102.68
dtype: float64

#### Total Volume of Orders in Each Cluster 

In [21]:
# Calulating the total volume of orders each day for each cluster
clusterv = kite_wd.pivot_table(index = 'Cluster Label', 
                               columns = 'DateDespatched', 
                               values = 'Volume (m3)', aggfunc = 'sum')
clusterv

DateDespatched,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-03,2023-04-04,2023-04-05,2023-04-06,2023-04-11,2023-04-12,2023-04-13,2023-04-14,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-24
Cluster Label,,,,,,,,,,,,,,,,,,,
0,7.251909,9.952171,20.547438,7.028912,13.674806,12.504105,16.175590,25.524440,22.823313,9.004243,9.513334,12.239129,15.541301,8.406201,12.650169,21.883134,9.047948,11.854074,0.960720
1,63.010366,50.921280,44.408904,50.590373,61.112631,79.606859,83.820526,62.934045,57.064986,100.333407,65.069227,91.774469,62.226217,40.791971,104.325235,41.491439,30.716260,51.041497,4.661103
2,45.287219,37.617849,41.897670,31.430588,90.849628,33.464670,58.023650,43.021609,27.872559,48.621494,44.647164,57.621417,25.759619,54.954580,34.986245,42.644347,52.564509,29.261809,2.694345
3,17.812642,15.381631,55.758238,39.402011,14.823694,34.384352,30.000494,43.844445,31.130046,25.445769,63.035226,29.894990,13.534090,38.842378,38.367388,25.399783,31.205333,26.378538,0.587527
4,23.800001,41.091753,26.616281,21.008913,44.097096,26.908873,17.102626,15.366825,17.169766,26.685753,31.696877,23.222990,29.163183,34.181551,43.853590,23.832307,19.378556,20.252817,NaN


In [22]:
# Finding the median total volume of orders each day for each cluster
clusterv1 = clusterv.median(axis=1)
clusterv1

Cluster Label
0    12.239129
1    61.112631
2    42.644347
3    30.000494
4    25.224294
dtype: float64

In [23]:
# Adjusting this for the number of trucks needed
clusterv2 = clusterv1/trucks_per_cluster
clusterv2

Cluster Label
0    12.239129
1    30.556315
2    21.322173
3    15.000247
4    25.224294
dtype: float64

#### Total Pallets for Orders in Each Cluster 

In [24]:
# Calulating the total pallets for orders each day for each cluster
clusterp = kite_wd.pivot_table(index = 'Cluster Label', 
                               columns = 'DateDespatched', 
                               values = 'Pallets', aggfunc = 'sum')
clusterp

DateDespatched,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-03,2023-04-04,2023-04-05,2023-04-06,2023-04-11,2023-04-12,2023-04-13,2023-04-14,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-24
Cluster Label,,,,,,,,,,,,,,,,,,,
0,4.626825,5.520840,15.878389,4.072352,8.579204,8.750306,11.926882,16.514156,9.632256,6.564483,6.806422,10.289356,10.948243,5.453371,8.497022,14.055280,7.252378,7.704178,0.867907
1,38.252106,30.097933,25.875409,30.604619,34.749950,52.090649,59.073879,37.945566,35.859555,64.183745,38.207957,61.017147,40.610650,26.565372,60.519807,29.744151,19.234192,34.790947,4.817651
2,25.430281,26.711982,28.232383,20.638080,59.148193,25.695971,37.088422,27.698291,22.694177,30.436707,30.523396,34.703450,18.132844,31.487958,22.920581,27.114326,35.655519,21.307101,1.257240
3,11.528200,12.985316,36.069616,30.100167,9.401686,23.829074,21.668728,27.415797,25.247212,20.095868,33.847965,17.274826,12.430367,26.191024,23.686885,16.836916,18.823916,18.991872,0.523323
4,15.194340,22.966426,18.886072,14.503915,26.996616,15.696868,10.335742,11.650372,10.634615,17.970220,18.110261,13.856994,23.396730,17.558513,25.580183,14.777550,8.970153,16.775009,NaN


In [25]:
# Finding the median total pallets for orders each day for each cluster
clusterp1 = clusterp.median(axis=1)
clusterp1

Cluster Label
0     8.497022
1    35.859555
2    27.114326
3    20.095868
4    16.235939
dtype: float64

In [26]:
# Adjusting this for the number of trucks needed
clusterp2 = clusterp1/trucks_per_cluster
clusterp2

Cluster Label
0     8.497022
1    17.929778
2    13.557163
3    10.047934
4    16.235939
dtype: float64

#### Joining Total Weight, Volume, and Number of Pallets for Each Cluster

In [27]:
# Concatenating clusterw2, clusterv2, and clusterp2 on the basis of Cluster Label
clusterstats = pd.concat([clusterw2, clusterv2, clusterp2], axis=1)

# Changing column names
clusterstats.columns = ['Weight', 'Volume (m3)', 'Pallets']

# Viewing this DataFrame
clusterstats

,Weight,Volume (m3),Pallets
Cluster Label,,,
0,1329.11,12.239129,8.497022
1,3042.61,30.556315,17.929778
2,2284.89,21.322173,13.557163
3,1385.05,15.000247,10.047934
4,2102.68,25.224294,16.235939


# 

### Investigating the Types of Trucks Needed in Each Cluster (Median)

#### Trucks Needed in Cluster 0

In [28]:
# Extracting the median total weight, volume, and number of pallets used on a single day in cluster 0
c0 = clusterstats.iloc[0]
c0

Weight         1329.110000
Volume (m3)      12.239129
Pallets           8.497022
Name: 0, dtype: float64

In [29]:
# Computing the type of trucks needed in cluster 0
trucks_needed(c0)

The 7.5 - tonne truck is needed.


#### Trucks Needed in Cluster 1

In [30]:
# Extracting the median total weight, volume, and number of pallets used on a single day in cluster 1
c1 = clusterstats.iloc[1]
c1

Weight         3042.610000
Volume (m3)      30.556315
Pallets          17.929778
Name: 1, dtype: float64

In [31]:
# Computing the type of trucks needed in cluster 1
trucks_needed(c1)

The 18 - tonne truck is needed.


#### Trucks Needed in Cluster 2

In [32]:
# Extracting the median total weight, volume, and number of pallets used on a single day in cluster 2
c2 = clusterstats.iloc[2]
c2

Weight         2284.890000
Volume (m3)      21.322173
Pallets          13.557163
Name: 2, dtype: float64

In [33]:
# Computing the type of trucks needed in cluster 2
trucks_needed(c2)

The 7.5 - tonne truck is needed.


#### Trucks Needed in Cluster 3

In [34]:
# Extracting the median total weight, volume, and number of pallets used on a single day in cluster 3
c3 = clusterstats.iloc[3]
c3

Weight         1385.050000
Volume (m3)      15.000247
Pallets          10.047934
Name: 3, dtype: float64

In [35]:
# Computing the type of trucks needed in cluster 3
trucks_needed(c3)

The 7.5 - tonne truck is needed.


#### Trucks Needed in Cluster 4

In [36]:
# Extracting the median total weight, volume, and number of pallets used on a single day in cluster 4
c4 = clusterstats.iloc[4]
c4

Weight         2102.680000
Volume (m3)      25.224294
Pallets          16.235939
Name: 4, dtype: float64

In [37]:
# Computing the type of trucks needed in cluster 4
trucks_needed(c4)

The 18 - tonne truck is needed.


# 

### Assigning Cluster 0 with One 7.5-Tonne LGV

In [38]:
# Subsetting the data to only look at cluster 0
c0_df = kite_wd[kite_wd['Cluster Label'] == 0]

# Checking this has worked
c0_df['Cluster Label'].unique()

array([0], dtype=int64)

In [39]:
# Viewing the DataFrame
c0_df.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
649,19.50,2280694,50656,2023-03-31,DX Freight,284.40,0.00,2023-03-31,27.72,63.0,4.70400,2.500000,118,B96,52.264208,-1.973759,0,4
650,19.50,2299385,901624,2023-04-22,MFS,226.95,0.00,2023-04-24,34.46,125.0,0.03136,0.208333,118,B96,52.264208,-1.973759,0,0
980,26.37,2281195,63911,2023-03-31,DX Freight,93.48,12.34,2023-03-31,11.02,18.2,0.33824,0.333333,2907,WR9,52.272714,-2.154912,0,4
981,26.37,2294928,63911,2023-04-18,DX Freight,223.56,0.00,2023-04-18,14.00,27.6,1.48840,1.000000,2907,WR9,52.272714,-2.154912,0,1
982,26.37,2291845,413108,2023-04-14,DX Freight,289.18,0.00,2023-04-14,38.65,91.0,0.68486,0.649074,2907,WR9,52.272714,-2.154912,0,4


In [40]:
# Calculating the total weight, volume, and number of pallets in cluster 0 for each day
capacity_c0 = c0_df.groupby(['DateDespatched'])['Weight', 'Volume (m3)', 'Pallets'].sum()
capacity_c0

,Weight,Volume (m3),Pallets
DateDespatched,,,
2023-03-27,861.03,7.251909,4.626825
2023-03-28,453.91,9.952171,5.520840
2023-03-29,2345.64,20.547438,15.878389
2023-03-30,652.47,7.028912,4.072352
2023-03-31,934.83,13.674806,8.579204
2023-04-03,971.10,12.504105,8.750306
2023-04-04,1934.39,16.175590,11.926882
2023-04-05,2436.08,25.524440,16.514156
2023-04-06,2117.81,22.823313,9.632256


In [41]:
# Checking that the weight each day is within the capacity of the 7.5-tonne LGV
capacity_c0['Weight_Capacity'] = capacity_c0['Weight']/4000

# Checking that the volume each day is within the capacity of the 7.5-tonne LGV
capacity_c0['Volume_Capacity'] = capacity_c0['Volume (m3)']/25

# Checking that the volume each day is within the capacity of the 7.5-tonne LGV
capacity_c0['Pallet_Capacity'] = capacity_c0['Pallets']/14

# Viewing these new columns
capacity_c0

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,861.03,7.251909,4.626825,0.215257,0.290076,0.330488
2023-03-28,453.91,9.952171,5.520840,0.113477,0.398087,0.394346
2023-03-29,2345.64,20.547438,15.878389,0.586410,0.821898,1.134171
2023-03-30,652.47,7.028912,4.072352,0.163117,0.281156,0.290882
2023-03-31,934.83,13.674806,8.579204,0.233708,0.546992,0.612800
2023-04-03,971.10,12.504105,8.750306,0.242775,0.500164,0.625022
2023-04-04,1934.39,16.175590,11.926882,0.483598,0.647024,0.851920
2023-04-05,2436.08,25.524440,16.514156,0.609020,1.020978,1.179583
2023-04-06,2117.81,22.823313,9.632256,0.529452,0.912933,0.688018


In [42]:
# Identifying days when the orders have run over the capacity of the 7.5-tonne LGV
c0_overcap = capacity_c0[(capacity_c0['Weight_Capacity'] > 1) | (capacity_c0['Volume_Capacity'] > 1) | (capacity_c0['Pallet_Capacity'] > 1)]
c0_overcap

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-29,2345.64,20.547438,15.878389,0.586410,0.821898,1.134171
2023-04-05,2436.08,25.524440,16.514156,0.609020,1.020978,1.179583
2023-04-19,2255.06,21.883134,14.055280,0.563765,0.875325,1.003949


In [43]:
# What percentage of days are running over capacity of the 7.5-tonne LGV
len(c0_overcap)/len(capacity_c0)

0.15789473684210525

It appears that on 16% of days, the orders exceed the capacity of the 7.5-tonne LGV.

# 

### Assigning Cluster 1 with Two 18-Tonne HGVs

In [44]:
# Subsetting the data to only look at cluster 1
c1_df = kite_wd[kite_wd['Cluster Label'] == 1]

# Checking this has worked
c1_df['Cluster Label'].unique()

array([1], dtype=int64)

In [45]:
# Viewing the DataFrame
c1_df.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
0,26.84,2295440,7700,2023-04-18,MFS,375.40,0.0,2023-04-18,34.46,110.00,0.2250,0.200000,766,DY6,52.496759,-2.173682,1,1
1,26.84,2277415,67940,2023-03-28,DX Freight,236.61,0.0,2023-03-28,16.45,33.30,0.9590,0.473039,766,DY6,52.496759,-2.173682,1,1
2,26.84,2279524,67940,2023-03-30,DX Freight,70.58,13.2,2023-03-30,11.79,21.80,0.2091,0.222222,766,DY6,52.496759,-2.173682,1,3
5,26.84,2292112,778040,2023-04-14,DX Freight,104.00,0.0,2023-04-14,20.55,44.20,0.1235,0.054167,766,DY6,52.496759,-2.173682,1,4
6,26.84,2298627,846996,2023-04-21,DX Freight,111.36,0.0,2023-04-21,23.25,51.12,0.1296,0.100000,766,DY6,52.496759,-2.173682,1,4


In [46]:
# Calculating the total weight, volume, and number of pallets in cluster 1 for each day
capacity_c1 = c1_df.groupby(['DateDespatched'])['Weight', 'Volume (m3)', 'Pallets'].sum()
capacity_c1

,Weight,Volume (m3),Pallets
DateDespatched,,,
2023-03-27,6085.22,63.010366,38.252106
2023-03-28,4655.93,50.921280,30.097933
2023-03-29,3643.41,44.408904,25.875409
2023-03-30,5503.92,50.590373,30.604619
2023-03-31,6545.02,61.112631,34.749950
2023-04-03,8274.88,79.606859,52.090649
2023-04-04,8492.52,83.820526,59.073879
2023-04-05,7210.06,62.934045,37.945566
2023-04-06,3823.51,57.064986,35.859555


In [47]:
# Checking that the weight each day is within the capacity of two 18-tonne HGVs
capacity_c1['Weight_Capacity'] = capacity_c1['Weight']/20000

# Checking that the volume each day is within the capacity of two 18-tonne HGVs
capacity_c1['Volume_Capacity'] = capacity_c1['Volume (m3)']/90

# Checking that the volume each day is within the capacity of two 18-tonne HGVs
capacity_c1['Pallet_Capacity'] = capacity_c1['Pallets']/48

# Viewing these new columns
capacity_c1

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,6085.22,63.010366,38.252106,0.304261,0.700115,0.796919
2023-03-28,4655.93,50.921280,30.097933,0.232797,0.565792,0.627040
2023-03-29,3643.41,44.408904,25.875409,0.182170,0.493432,0.539071
2023-03-30,5503.92,50.590373,30.604619,0.275196,0.562115,0.637596
2023-03-31,6545.02,61.112631,34.749950,0.327251,0.679029,0.723957
2023-04-03,8274.88,79.606859,52.090649,0.413744,0.884521,1.085222
2023-04-04,8492.52,83.820526,59.073879,0.424626,0.931339,1.230706
2023-04-05,7210.06,62.934045,37.945566,0.360503,0.699267,0.790533
2023-04-06,3823.51,57.064986,35.859555,0.191175,0.634055,0.747074


In [48]:
# Identifying days when the orders have run over the capacity of two 18-tonne HGVs
c1_overcap = capacity_c1[(capacity_c1['Weight_Capacity'] > 1) | (capacity_c1['Volume_Capacity'] > 1) | (capacity_c1['Pallet_Capacity'] > 1)]
c1_overcap

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-04-03,8274.88,79.606859,52.090649,0.413744,0.884521,1.085222
2023-04-04,8492.52,83.820526,59.073879,0.424626,0.931339,1.230706
2023-04-11,8140.87,100.333407,64.183745,0.407044,1.114816,1.337161
2023-04-13,12033.06,91.774469,61.017147,0.601653,1.019716,1.271191
2023-04-18,13017.81,104.325235,60.519807,0.650890,1.159169,1.260829


In [49]:
# What percentage of days are running over capacity of the two 18-tonne HGVs
len(c1_overcap)/len(capacity_c1)

0.2631578947368421

It appears that on 26% of days, the orders exceed the capacity of the two 18-tonne HGVs.

# 

### Assigning Cluster 2 with Two 7.5-Tonne LGVs

In [50]:
# Subsetting the data to only look at cluster 2
c2_df = kite_wd[kite_wd['Cluster Label'] == 2]

# Checking this has worked
c2_df['Cluster Label'].unique()

array([2], dtype=int64)

In [51]:
# Viewing the DataFrame
c2_df.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
86,42.25,2291876,139982,2023-04-14,DX Freight,143.97,0.0,2023-04-14,13.23,25.78,0.610060,0.330357,1820,NG8,52.964498,-1.213406,2,4
87,42.25,2284231,546053,2023-04-04,DX Freight,282.38,0.0,2023-04-04,22.19,48.44,4.711619,2.506349,1820,NG8,52.964498,-1.213406,2,1
88,42.25,2291035,599677,2023-04-13,MFS,346.80,0.0,2023-04-13,39.15,130.80,0.197500,0.166667,1820,NG8,52.964498,-1.213406,2,3
89,42.25,2297741,608200,2023-04-20,MFS,188.16,0.0,2023-04-20,39.15,94.32,0.168000,0.133333,1820,NG8,52.964498,-1.213406,2,3
90,42.25,2277978,621400,2023-03-29,DX Freight,230.80,0.0,2023-03-29,19.95,42.76,0.416000,0.197895,1820,NG8,52.964498,-1.213406,2,2


In [52]:
# Calculating the total weight, volume, and number of pallets in cluster 2 for each day
capacity_c2 = c2_df.groupby(['DateDespatched'])['Weight', 'Volume (m3)', 'Pallets'].sum()
capacity_c2

,Weight,Volume (m3),Pallets
DateDespatched,,,
2023-03-27,4469.25,45.287219,25.430281
2023-03-28,5724.00,37.617849,26.711982
2023-03-29,5086.67,41.897670,28.232383
2023-03-30,3097.53,31.430588,20.638080
2023-03-31,10774.05,90.849628,59.148193
2023-04-03,4171.57,33.464670,25.695971
2023-04-04,4052.52,58.023650,37.088422
2023-04-05,4837.88,43.021609,27.698291
2023-04-06,4569.78,27.872559,22.694177


In [53]:
# Checking that the weight each day is within the capacity of two 7.5-tonne LGVs
capacity_c2['Weight_Capacity'] = capacity_c2['Weight']/8000

# Checking that the volume each day is within the capacity of two 7.5-tonne LGVs
capacity_c2['Volume_Capacity'] = capacity_c2['Volume (m3)']/50

# Checking that the volume each day is within the capacity of two 7.5-tonne LGVs
capacity_c2['Pallet_Capacity'] = capacity_c2['Pallets']/24

# Viewing these new columns
capacity_c2

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,4469.25,45.287219,25.430281,0.558656,0.905744,1.059595
2023-03-28,5724.00,37.617849,26.711982,0.715500,0.752357,1.112999
2023-03-29,5086.67,41.897670,28.232383,0.635834,0.837953,1.176349
2023-03-30,3097.53,31.430588,20.638080,0.387191,0.628612,0.859920
2023-03-31,10774.05,90.849628,59.148193,1.346756,1.816993,2.464508
2023-04-03,4171.57,33.464670,25.695971,0.521446,0.669293,1.070665
2023-04-04,4052.52,58.023650,37.088422,0.506565,1.160473,1.545351
2023-04-05,4837.88,43.021609,27.698291,0.604735,0.860432,1.154095
2023-04-06,4569.78,27.872559,22.694177,0.571222,0.557451,0.945591


In [54]:
# Identifying days when the orders have run over the capacity of two 7.5-tonne LGVs
c2_overcap = capacity_c2[(capacity_c2['Weight_Capacity'] > 1) | (capacity_c2['Volume_Capacity'] > 1) | (capacity_c2['Pallet_Capacity'] > 1)]
c2_overcap

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,4469.25,45.287219,25.430281,0.558656,0.905744,1.059595
2023-03-28,5724.00,37.617849,26.711982,0.715500,0.752357,1.112999
2023-03-29,5086.67,41.897670,28.232383,0.635834,0.837953,1.176349
2023-03-31,10774.05,90.849628,59.148193,1.346756,1.816993,2.464508
2023-04-03,4171.57,33.464670,25.695971,0.521446,0.669293,1.070665
2023-04-04,4052.52,58.023650,37.088422,0.506565,1.160473,1.545351
2023-04-05,4837.88,43.021609,27.698291,0.604735,0.860432,1.154095
2023-04-11,4531.13,48.621494,30.436707,0.566391,0.972430,1.268196
2023-04-12,5784.69,44.647164,30.523396,0.723086,0.892943,1.271808


In [55]:
# What percentage of days are running over capacity of the two 7.5-tonne LGVs
len(c2_overcap)/len(capacity_c2)

0.6842105263157895

It appears that on 68% of days, the orders exceed the capacity of the two 7.5-tonne LGVs.

# 

### Assigning Cluster 3 with Two 7.5-Tonne LGVs

In [56]:
# Subsetting the data to only look at cluster 3
c3_df = kite_wd[kite_wd['Cluster Label'] == 3]

# Checking this has worked
c3_df['Cluster Label'].unique()

array([3], dtype=int64)

In [57]:
# Viewing the DataFrame
c3_df.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
11,43.04,2292996,9759,2023-04-16,DX Freight,148.41,0.0,2023-04-17,14.77,29.24,0.219678,0.200025,1934,OX18,51.767526,-1.582311,3,0
12,43.04,2280292,99458,2023-03-30,MFS,425.52,0.0,2023-03-31,40.07,75.60,2.095200,1.000000,1934,OX18,51.767526,-1.582311,3,4
13,43.04,2298694,99458,2023-04-21,MFS,649.20,0.0,2023-04-21,78.54,148.60,2.191200,1.106667,1934,OX18,51.767526,-1.582311,3,4
15,43.04,2297271,524321,2023-04-20,DX Freight,677.11,0.0,2023-04-20,47.89,115.93,1.679950,0.896429,1934,OX18,51.767526,-1.582311,3,3
16,43.04,2284166,659011,2023-04-04,MFS,252.96,0.0,2023-04-04,40.07,144.48,1.380000,1.000000,1934,OX18,51.767526,-1.582311,3,1


In [58]:
# Calculating the total weight, volume, and number of pallets in cluster 3 for each day
capacity_c3 = c3_df.groupby(['DateDespatched'])['Weight', 'Volume (m3)', 'Pallets'].sum()
capacity_c3

,Weight,Volume (m3),Pallets
DateDespatched,,,
2023-03-27,1901.76,17.812642,11.528200
2023-03-28,2020.44,15.381631,12.985316
2023-03-29,3506.48,55.758238,36.069616
2023-03-30,3009.75,39.402011,30.100167
2023-03-31,788.30,14.823694,9.401686
2023-04-03,2841.85,34.384352,23.829074
2023-04-04,3029.89,30.000494,21.668728
2023-04-05,3824.49,43.844445,27.415797
2023-04-06,4986.93,31.130046,25.247212


In [59]:
# Checking that the weight each day is within the capacity of two 7.5-tonne LGVs
capacity_c3['Weight_Capacity'] = capacity_c3['Weight']/8000

# Checking that the volume each day is within the capacity of two 7.5-tonne LGVs
capacity_c3['Volume_Capacity'] = capacity_c3['Volume (m3)']/50

# Checking that the volume each day is within the capacity of two 7.5-tonne LGVs
capacity_c3['Pallet_Capacity'] = capacity_c3['Pallets']/24

# Viewing these new columns
capacity_c3

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,1901.76,17.812642,11.528200,0.237720,0.356253,0.480342
2023-03-28,2020.44,15.381631,12.985316,0.252555,0.307633,0.541055
2023-03-29,3506.48,55.758238,36.069616,0.438310,1.115165,1.502901
2023-03-30,3009.75,39.402011,30.100167,0.376219,0.788040,1.254174
2023-03-31,788.30,14.823694,9.401686,0.098538,0.296474,0.391737
2023-04-03,2841.85,34.384352,23.829074,0.355231,0.687687,0.992878
2023-04-04,3029.89,30.000494,21.668728,0.378736,0.600010,0.902864
2023-04-05,3824.49,43.844445,27.415797,0.478061,0.876889,1.142325
2023-04-06,4986.93,31.130046,25.247212,0.623366,0.622601,1.051967


In [60]:
# Identifying days when the orders have run over the capacity of two 7.5-tonne LGVs
c3_overcap = capacity_c3[(capacity_c3['Weight_Capacity'] > 1) | (capacity_c3['Volume_Capacity'] > 1) | (capacity_c3['Pallet_Capacity'] > 1)]
c3_overcap

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-29,3506.48,55.758238,36.069616,0.438310,1.115165,1.502901
2023-03-30,3009.75,39.402011,30.100167,0.376219,0.788040,1.254174
2023-04-05,3824.49,43.844445,27.415797,0.478061,0.876889,1.142325
2023-04-06,4986.93,31.130046,25.247212,0.623366,0.622601,1.051967
2023-04-12,5845.49,63.035226,33.847965,0.730686,1.260705,1.410332
2023-04-17,3710.51,38.842378,26.191024,0.463814,0.776848,1.091293


In [61]:
# What percentage of days are running over capacity of the two 7.5-tonne LGVs
len(c3_overcap)/len(capacity_c3)

0.3157894736842105

It appears that on 32% of days, the orders exceed the capacity of the two 7.5-tonne LGVs.

# 

### Assigning Cluster 4 with One 18-Tonne HGV

In [62]:
# Subsetting the data to only look at cluster 4
c4_df = kite_wd[kite_wd['Cluster Label'] == 4]

# Checking this has worked
c4_df['Cluster Label'].unique()

array([4], dtype=int64)

In [63]:
# Viewing the DataFrame
c4_df.head()

,distance,OrderID,WebsiteID,OrderDate,Courier,ProductsExVAT,DeliveryExVAT,DateDespatched,DeliveryCost,Weight,Volume (m3),Pallets,id,Outward Code,latitude,longitude,Cluster Label,weekday
64,44.21,2281052,16152,2023-03-31,DX Freight,124.36,0.00,2023-03-31,22.28,48.74,0.090629,0.059524,1664,MK1,52.006042,-0.726102,4,4
65,44.21,2276229,69001,2023-03-27,DX Freight,48.68,12.34,2023-03-27,11.02,6.00,0.400412,0.263889,1664,MK1,52.006042,-0.726102,4,0
67,44.21,2289226,378832,2023-04-12,MFS,441.28,0.00,2023-04-12,76.73,195.56,2.454680,1.371429,1664,MK1,52.006042,-0.726102,4,2
69,44.21,2287705,494355,2023-04-11,DX Freight,78.60,14.02,2023-04-11,12.52,23.54,0.349940,0.395000,1664,MK1,52.006042,-0.726102,4,1
70,44.21,2277715,562226,2023-03-28,MFS,425.20,0.00,2023-03-28,39.15,135.80,0.999000,0.952381,1664,MK1,52.006042,-0.726102,4,1


In [64]:
# Calculating the total weight, volume, and number of pallets in cluster 4 for each day
capacity_c4 = c4_df.groupby(['DateDespatched'])['Weight', 'Volume (m3)', 'Pallets'].sum()
capacity_c4

,Weight,Volume (m3),Pallets
DateDespatched,,,
2023-03-27,1615.05,23.800001,15.194340
2023-03-28,4781.02,41.091753,22.966426
2023-03-29,2333.34,26.616281,18.886072
2023-03-30,3407.72,21.008913,14.503915
2023-03-31,2427.28,44.097096,26.996616
2023-04-03,2212.35,26.908873,15.696868
2023-04-04,1783.50,17.102626,10.335742
2023-04-05,1002.20,15.366825,11.650372
2023-04-06,1047.23,17.169766,10.634615


In [65]:
# Checking that the weight each day is within the capacity of the 18-tonne HGV
capacity_c4['Weight_Capacity'] = capacity_c4['Weight']/10000

# Checking that the volume each day is within the capacity of the 18-tonne HGV
capacity_c4['Volume_Capacity'] = capacity_c4['Volume (m3)']/45

# Checking that the volume each day is within the capacity of the 18-tonne HGV
capacity_c4['Pallet_Capacity'] = capacity_c4['Pallets']/24

# Viewing these new columns
capacity_c4

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-27,1615.05,23.800001,15.194340,0.161505,0.528889,0.633098
2023-03-28,4781.02,41.091753,22.966426,0.478102,0.913150,0.956934
2023-03-29,2333.34,26.616281,18.886072,0.233334,0.591473,0.786920
2023-03-30,3407.72,21.008913,14.503915,0.340772,0.466865,0.604330
2023-03-31,2427.28,44.097096,26.996616,0.242728,0.979935,1.124859
2023-04-03,2212.35,26.908873,15.696868,0.221235,0.597975,0.654036
2023-04-04,1783.50,17.102626,10.335742,0.178350,0.380058,0.430656
2023-04-05,1002.20,15.366825,11.650372,0.100220,0.341485,0.485432
2023-04-06,1047.23,17.169766,10.634615,0.104723,0.381550,0.443109


In [66]:
# Identifying days when the orders have run over the capacity of the 18-tonne HGV
c4_overcap = capacity_c4[(capacity_c4['Weight_Capacity'] > 1) | (capacity_c4['Volume_Capacity'] > 1) | (capacity_c4['Pallet_Capacity'] > 1)]
c4_overcap

,Weight,Volume (m3),Pallets,Weight_Capacity,Volume_Capacity,Pallet_Capacity
DateDespatched,,,,,,
2023-03-31,2427.28,44.097096,26.996616,0.242728,0.979935,1.124859
2023-04-18,3990.24,43.853590,25.580183,0.399024,0.974524,1.065841


In [67]:
# What percentage of days are running over capacity of the 18-tonne HGV
len(c4_overcap)/len(capacity_c4)

0.1111111111111111

It appears that on 11% of days, the orders exceed the capacity of the 18-tonne HGV.